In [5]:
import langchain

In [23]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 unstructured

^C
Note: you may need to restart the kernel to use updated packages.


In [36]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_chroma import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain import hub
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser



os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
model = ChatOpenAI(model="gpt-3.5-turbo")



In [37]:
path = r"C:\Users\reply\Documents\GitHub\RepoAgent\markdown_docs\repo_agent"
loader = DirectoryLoader(path, glob="./*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
docs = loader.load()

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:11<00:00,  1.20it/s]


In [38]:
def split_documents(doc, chunk_size=250, chunk_overlap=30):
    headers_to_split_on = [
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    # Split
    splits = text_splitter.split_documents(md_header_splits)
   
    return splits

In [39]:
all_splits = []
for doc in docs:
    splits = split_documents(doc)
    all_splits.extend(splits)


In [40]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)



In [41]:
rag_chain.invoke("Which are the patameters of observe_updating")

'The observe_updating function does not take any parameters.'

In [43]:
rag_chain.invoke("how get_staged_pys works?")

'The get_staged_pys function uses GitPython to detect newly added Python files in the staging area. It compares the staging area with the original HEAD commit to identify these changes. This function is essential for tracking staged changes for Python files in a Git repository.'

In [42]:
rag_chain.invoke("What last.md does?")

'Last.md saves and displays the markdown content of an existing file after making structural changes.'

In [45]:
rag_chain.invoke("What last.md does?")

'last.md saves and updates the markdown content of an existing file after making structural changes.'

In [46]:
rag_chain.invoke("What prova.md does?")

'The prova.md function assigns values to variables a, b, and c, and then returns the values of b and c. It does not take any parameters and is used for version control operations with the git library.'